In [1]:
import subprocess
import numpy as np
import os
import sys
sys.path.append(f'{os.environ["HOME"]}/Projects/planckClusters/catalogs')
from load_catalogs import load_PSZcatalog
from astropy.table import Table
from astropy.io import fits
from astropy import wcs
import astropy.units as u
from glob import glob
import shlex
                
# parallel processor
from utilities import parallel_process

In [2]:
# this allows us to run the pipeline not interactively. 
os.environ['HEADASNOQUERY'] = ''
os.environ['HEADASPROMPT'] = '/dev/null'
# os.environ['PFILES'] = '/tmp/$$.tmp/pfiles;$HEADAS/syspfiles'

In [4]:
def runnh(ra, dec):
    '''This function will automatically run the ftool nh
    on a given RA and Dec in J2000 equinox coordinates'''

    nh = 0
    cmd = f'nh equinox=2000 ra={ra} dec={dec}'
    args = shlex.split(cmd)
    out = ""

    with subprocess.Popen(args,
                          stdout=subprocess.PIPE,
                          universal_newlines=True) as proc:

        proc.wait()
        out = proc.stdout.read()

    outlines = out.split("\n")
    for line in outlines:
        if "Weighted average" in line:
            nh = line.split(" ")[-1]

    return out, float(nh)

def GetLuminosity(flux, distance):
        #assumes distance is in mpc
        #converts to cm
        #returns L = (4piD^2)*f
        distance = 3.86E24 * distance
        return 4*math.pi*distance*distance*flux

In [5]:
def get_immediate_subdirectories(a_dir):
    ''' Get a list of a directorys immediate subdirectories'''
    return [os.path.join(a_dir, name) for name in os.listdir(a_dir)
            if os.path.isdir(os.path.join(a_dir, name))]

def get_immediate_subfiles(a_dir):
    ''' Get a list of all the FILES in a directory'''
    return [os.path.join(a_dir, name) for name in os.listdir(a_dir)
            if os.path.isfile(os.path.join(a_dir, name))]

def check_exe(exe, verb=False):
    ''' Checks to make sure we have the appropriate system command available.
    If we don't it raises an exception.

    '''

    path = os.environ['PATH'].split(':')
    for p in path:
        f = os.path.join(p, exe)
        if os.path.isfile(f):
            if verb:
                print("# Found %s in %s" % (exe, f), file=sys.stderr)
            return True
    if verb:
        # it wasn't found
        print("# ERROR: Couldn't find %s" % exe, file=sys.stderr)
    raise FileNotFoundError(exe)
    return False

def set_env():
    os.environ['PFILES'] = f"/tmp/{os.getpid()}.tmp/pfiles;$HEADAS/syspfiles"
    pfiles = f'/tmp/{os.getpid()}.tmp/pfiles'
    if not os.path.isdir(pfiles):
        os.makedirs(pfiles)

In [6]:
def inv_beta_model(So, rc, beta, bkg):
    ''' Solves for where (radius) the beta model is equal to the background
    
    Make sure you keep track of the units going into all this. You wanna make sure the radius comes
    out as something that makes sense.

    '''
    
    a = -3 * beta + 0.5 
    
    b = (bkg/So)**(1/a) - 1
    
    c = b * rc**2
    
    return np.sqrt(c)

In [7]:
def create_bkg_regions(name, outpath):
    ''' For each obs id we are going to create a background region. 10.5 arcmin
    centered on the pointing position of each observation. 
    
    '''
    
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')
    
    # find the event files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw[1-4]po_cl.evt', recursive=True)
    
    if len(files) < 1:
        return name
    
    # create a place for the products
    if not os.path.isdir(f'{outpath}/{name}/spec_files'):
        os.makedirs(f'{outpath}/{name}/spec_files')
    
    # write one background region for each observation
    for f_in in files:
        
        # get pointing
        point_ra = fits.getheader(f_in, ext=0)['RA_PNT']
        point_dec = fits.getheader(f_in, ext=0)['DEC_PNT']
        
        # we cannot use the X/Y position in the catalog because it was detected in 
        # the combined frame. We need to use the RA/DEC and then find the pixel positions
        # in the individual frames. Lame.
        # WCS info to convert world coords to pixels 
        hdr = fits.getheader(f_in.replace('cl.evt', 'ex.img'))
        w = wcs.WCS(hdr)
        pixels = w.wcs_world2pix(point_ra, point_dec, 0)
        
        obs_id = f_in.split('/')[-2]
        
        # write out the regions
        with open(f'{outpath}/{name}/spec_files/{obs_id}_bkg.reg', 'w') as reg:     
            # write the background region
            reg.write(f"circle({pixels[0]:.5f},{pixels[1]:.5f},{(11 * 60 /2.36):.5f})\n")

            for j, xc, yc, rc, rotc in detects[['INDEX', 'RA', 'DEC', 'R', 'ROTANG']]:
                pixels = w.wcs_world2pix(xc, yc, 0)
                reg.write(f'-ellipse({pixels[0]:.5f},{pixels[1]:.5f},{rc[0]:.3f},{rc[1]:.3f},{rotc:.3f})\n')

In [8]:
def create_src_regions(name, outpath):
    ''' For each obs id we are going to create a background region. 10.5 arcmin
    centered on the pointing position of each observation. 
    
    '''
    
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')
    
    # find the event files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw[1-4]po_cl.evt', recursive=True)
    
    if len(files) < 1:
        return name
    
    # create a place for the products
    if not os.path.isdir(f'{outpath}/{name}/spec_files'):
        os.makedirs(f'{outpath}/{name}/spec_files')
    
    # get the MCMC fits:
    if os.path.isfile(f'{outpath}/{name}/{name}_mcmcfits.txt'):
        mcmcfit = Table.read(f'{outpath}/{name}/{name}_mcmcfits.txt', format='ascii', header_start=0)
    else:
        raise FileNotFoundError(f'{outpath}/{name}/{name}_mcmcfits.txt')

    # write one background region for each observation
    for f_in in files:
        
        obs_id = f_in.split('/')[-2]
        
        # we cannot use the X/Y position in the catalog because it was detected in 
        # the combined frame. We need to use the RA/DEC and then find the pixel positions
        # in the individual frames. Lame.
        # WCS info to convert world coords to pixels 
        hdr = fits.getheader(f_in.replace('cl.evt', 'ex.img'))
        w = wcs.WCS(hdr)        
        
        # now we have to loop through the detections
        for j, xc, yc in detects[['INDEX', 'RA', 'DEC']]:
            
            # we didn't fit all the sources with the MCMC
            if j not in mcmcfit['ID']:
                continue
                
            # write out the regions
            with open(f'{outpath}/{name}/spec_files/{obs_id}_{j}.reg', 'w') as reg:
                # Here we are going to draw a circle with a radius where the beta model just fades into
                # the background. These values come from the MCMC fits done previously.
                radius = inv_beta_model(mcmcfit['So_50'][mcmcfit['ID'] == j],
                                        mcmcfit['rc_50'][mcmcfit['ID'] == j],
                                        mcmcfit['beta_50'][mcmcfit['ID'] == j],
                                        mcmcfit['bg_50'][mcmcfit['ID'] == j]
                                       )
            
                pixels = w.wcs_world2pix(xc, yc, 0)
            
                reg.write(f"circle({pixels[0]},{pixels[1]},{(radius[0] * 60 / 2.36):.3f})\n")
            
                # have to loop through them again for the masks
                for k, xc2, yc2, rc, rotc in detects[['INDEX', 'RA', 'DEC', 'R', 'ROTANG']]:
                    if not j == k:
                        pixels = w.wcs_world2pix(xc, yc, 0)
                        reg.write(f'-ellipse({pixels[0]:.5f},{pixels[1]:.5f},'
                                  f'{rc[0]:.3f},{rc[1]:.3f},{rotc:.3f})\n')

In [9]:
def xselect_pha(name, outpath):
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')
    
    # find the event files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw[1-4]po_cl.evt', recursive=True)
    
    if len(files) < 1:
        return name
    
    # write one background region for each observation
    for f_in in files:
        
        obs_id = f_in.split('/')[-2]
        
        # create the background xselect call first
        if not os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_bkg.reg'):
            continue
    
        # write background xsel.in
        with open(f'{outpath}/{name}/spec_files/{obs_id}_bkg.in', 'w') as f:
            # this is the session name... specify random letters to run many at once.
            f.writelines(f'{obs_id}\n')
            f.writelines('read events\n')
            # set the data directory
            f.writelines('/'.join(f_in.split('/')[:5]) + '\n')
            # first entry
            f.writelines('/'.join(f_in.split('/')[5:]) + '\n')
            f.writelines('yes\n')
            f.writelines(f'filter region {outpath}/{name}/spec_files/{obs_id}_bkg.reg\n')
            f.writelines('extract spectrum\n')
            f.writelines(f'save spectrum {outpath}/{name}/spec_files/{obs_id}_bkg_pc.pha\n')
            if os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_bkg_pc.pha'):
                f.writelines('yes\n')
            f.writelines('exit\n')
            f.writelines('no\n')
        
        # log the output
        log_file = f'{outpath}/{name}/spec_files/{obs_id}_bkg_pha.log'
                         
        # call xselect
        check_exe('xselect')
        os.system(f'xselect < {outpath}/{name}/spec_files/{obs_id}_bkg.in > {log_file}')
        
        # now we do all the sources.
        for j in detects['INDEX']:
            if not os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_{j}.reg'):
                continue
                
            # write source xsel.in
            with open(f'{outpath}/{name}/spec_files/{obs_id}_{j}.in', 'w') as f:
                # this is the session name... specify random letters to run many at once.
                f.writelines(f'{obs_id}_{j}\n')
                f.writelines('read events\n')
                # set the data directory
                f.writelines('/'.join(f_in.split('/')[:5]) + '\n')
                # first entry
                f.writelines('/'.join(f_in.split('/')[5:]) + '\n')
                f.writelines('yes\n')
                f.writelines(f'filter region {outpath}/{name}/spec_files/{obs_id}_{j}.reg\n')
                f.writelines('extract spectrum\n')
                f.writelines(f'save spectrum {outpath}/{name}/spec_files/{obs_id}_{j}_pc.pha\n')
                if os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.pha'):
                    f.writelines('yes\n')
                f.writelines('exit\n')
                f.writelines('no\n')
                
            # log the output
            log_file = f'{outpath}/{name}/spec_files/{obs_id}_{j}_pha.log'
                
            # call xselect
            os.system(f'xselect < {outpath}/{name}/spec_files/{obs_id}_{j}.in > {log_file}')
#             proc = subprocess.Popen(f'xselect < {outpath}/{name}/spec_files/{obs_id}_{j}.in',
#                                 shell=True, preexec_fn=set_env)
#             proc.wait()

In [15]:
def mk_arf(name, outpath):
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')

    # find the event files
    files = glob(f'{outpath}/{name}/reduced/**/*xpcw[1-4]po_ex.img', recursive=True)
    
    if len(files) < 1:
        return name
    
    check_exe('xrtmkarf')
    
    for f_in in files:
        
        obs_id = f_in.split('/')[-2]
        # now we have to loop through the detections
        for j, ra, dec in detects[['INDEX', 'RA', 'DEC']]:
            # check that we have the files
            if not os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.pha'):
                continue
            else:
                pha = f'{outpath}/{name}/spec_files/{obs_id}_{j}_pc.pha'
    
            # we cannot use the X/Y position in the catalog because it was detected in 
            # the combined frame. We need to use the RA/DEC and then find the pixel positions
            # in the individual frames. Lame.
            # WCS info to convert world coords to pixels 
            hdr = fits.getheader(f_in)
            w = wcs.WCS(hdr)
            pixels = w.wcs_world2pix(ra, dec, 0)
    
            cmd = (f'xrtmkarf phafile={pha} srcx={pixels[0]} srcy={pixels[1]} '
                   f'outfile={outpath}/{name}/spec_files/{obs_id}_{j}_pc.arf '
                   f'psfflag=yes extended=yes expofile={f_in} clobber+')

            # log the output
            log_file = f'{outpath}/{name}/spec_files/{obs_id}_{j}_arf.log'

            # call
            print(cmd)
            #os.system(f'{cmd} > {log_file}')
            
            args = shlex.split(f'{cmd} > {log_file}')
            out = ""

            with subprocess.Popen(args,
                          stdout=subprocess.PIPE,
                          universal_newlines=True) as proc:

                proc.wait()
                out = proc.stdout.read()
            

In [19]:
def combine_spectra(name, outpath):
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')
    
    # get a list of the reduced obs.
    reduc = get_immediate_subdirectories(f'{outpath}/{name}/reduced')
    reduc_ids = [r.split('/')[-1] for r in reduc]
    
    if len(reduc) < 1:
        return
    
    check_exe('addascaspec')
    # we are gonna need to change directories
    wd = os.getcwd()
    
    # now we do all the sources.
    for j in detects['INDEX']:
        # make sure we have that source.
        obs_id = reduc_ids[0]
        if not os.path.isfile(f'{outpath}/{name}/spec_files/{obs_id}_{j}.reg'):
            continue
            
        # remove files if they are already there.
        if os.path.isfile(f'{outpath}/{name}/spec_files/{j}_pc.pha'):
            os.remove(f'{outpath}/{name}/spec_files/{j}_pc.pha')
        if os.path.isfile(f'{outpath}/{name}/spec_files/{j}_pc.arf'):
            os.remove(f'{outpath}/{name}/spec_files/{j}_pc.arf')
        if os.path.isfile(f'{outpath}/{name}/spec_files/{j}_bkg.pha'):
            os.remove(f'{outpath}/{name}/spec_files/{j}_bkg.pha')    
        
        # build the file lists:  
        spectra = [f'{obs_id}_{j}_pc.pha' for obs_id in reduc_ids]
        bkgs = [f'{obs_id}_bkg_pc.pha' for obs_id in reduc_ids]
        arfs = [f'{obs_id}_{j}_pc.arf' for obs_id in reduc_ids]  
        
        # the rmfs are harder to build
        rmfs = []
        for obs_id in reduc_ids:
            with open(f'{outpath}/{name}/spec_files/{obs_id}_{j}_arf.log', 'r') as f:
                lines = f.readlines()
                for l in lines:
                    if '/opt/caldb//data/swift/xrt/cpf/rmf/' in l:
                        rmfs.append(l.split("'")[1])
                
        # have to write the file list to pass to the spectrum combiner.
        with open(f'{outpath}/{name}/spec_files/{j}.list', 'w') as f:
            f.writelines(f'{" ".join(spectra)}\n')
            f.writelines(f'{" ".join(bkgs)}\n')
            f.writelines(f'{" ".join(arfs)}\n')
            f.writelines(f'{" ".join(rmfs)}\n')
        
        # log the output
        log_file = f'{j}_addascaspec.log'

        cmd = f'addascaspec {j}.list {j}_pc.pha {j}_pc.arf {j}_bkg.pha'
                         
        # call
#         print(cmd)
        os.chdir(f'{outpath}/{name}/spec_files/')
        os.system(f'{cmd} > {log_file}')
                         
        # do some other stuff -- add keywords to header
#         os.system(f'fparkey value={j}_pc.arf fitsfile={j}_pc.pha+1 keyword=ANCRFILE')
#         os.system(f'fparkey value={j}_bkg.pha fitsfile={j}_pc.pha+1 keyword=BACKFILE')
#         os.system(f'fparkey value={rmfs[0]} fitsfile={j}_pc.pha+1 keyword=RESPFILE')
                         
        # group the counts
        cnts = 10
        cmd = (f"grppha infile='{j}_pc.pha' outfile='{j}_pc_{cnts}cts.pha '
               f"chatter=0 comm='bad 1-31 & group min {cnts} & exit' clob+")
        args = shlex.split(cmd)
        out = ""

        with subprocess.Popen(args,
                          stdout=subprocess.PIPE,
                          universal_newlines=True) as proc:

            proc.wait()
            out = proc.stdout.read()

        os.chdir(wd)

#     return out, new_10cts_pha

In [71]:
def run_xspec_scripts(name, outpath):
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')
    
    
    # we are gonna need to change directories below
    wd = os.getcwd()
    os.chdir(f'{outpath}/{name}/spec_files/')
    
    # now we have to loop through the detections
    for j in detects['INDEX']:
        if not os.path.isfile(f'{j}_pc_{cnts}cts_xspec.in'):
            continue
        else:
            source_xspec_in = f'{j}_pc_{cnts}cts_xspec.in'
    
        # build the command and call it
        cmd = f"xspec - {source_xspec_in}"
        args = shlex.split(cmd)
        out = ""

        with subprocess.Popen(args,
                              stdout=subprocess.PIPE,
                              universal_newlines=True) as proc:

            try:
                proc.wait(timeout=5)
                out = proc.stdout.read()

                #Read the fluxes and norms from the stdout
                photon_flux = {}
                energy_flux = {}

                norms = []

                outlines = out.split('\n')
                for line in outlines:
                    if "Model Flux" in line:
                        band_low = line.split(" ")[8].split("(")[1]
                        band_high = line.split(" ")[10]
                        photons = float(line.split(" ")[3])
                        band = band_low + "-" + band_high
                        photon_flux[band] = photons

                        energy = float(line.split(" ")[5].split("(")[1])
                        energy_flux[band] = energy
                    elif "mekal" in line and "norm" in line:
                        #the norm will be two index before the +/- sign

                        pm_index = line.split(" ").index("+/-")
                        norms.append(line.split(" ")[pm_index - 2])
                    else:
                        pass

                    #Assuming the current script format, this is the one we want
                    #This is with PE Absorption
                goodnorm = float(norms[1])

                #Get the index of the line containing the sigmas
                conf_index = outlines.index(" Parameter   Confidence Range (1)")
                sigma_line = outlines[conf_index + 1]
                sigma_low = float(sigma_line.split(" ")[-1].split("(")[1].split(",")[0])
                sigma_high = float(sigma_line.split(" ")[-1].split("(")[1].split()",")[1].split(")")[0])
                sigma_av = (abs(sigma_low) + abs(sigma_high)) / 2

                SN = goodnorm / sigma_av
        #         print(out, energy_flux, photon_flux, goodnorm, sigma_av, SN)
            except:
                proc.kill()
                out = proc.stdout.read()
            os.chdir(wd)

            xspec_out, en_flux, phot_flux, norm, sigma, SN = \
                                            out, energy_flux, photon_flux, goodnorm, sigma_av, SN

            print("Energy Flux [ergs/cm^2/s]:\n", en_flux, '\n',
              "Photon Flux [photons]:\n", phot_flux, '\n', "norm: ", norm,
              '\n', "st. deviation: ", sigma, '\n', "S/N Ratio: ", SN,
              '\n')

def mk_xspec_scripts(name, outpath, kT=5, z=0.1):
    
    # if there aren't detections, don't bother doing anything.
    if os.path.isfile(f'{outpath}/{name}/{name}_vtp.detect'):
        detects = Table.read(f'{outpath}/{name}/{name}_vtp.detect', hdu=1)
    else:
        raise FileNotFoundError(f'{outpath}/{name}/{name}_vtp.detect')
    
    # now we have to loop through the detections
    for j, ra, dec in detects[['INDEX', 'RA', 'DEC']]:
        if not os.path.isfile(f'{outpath}/{name}/spec_files/{j}_pc_{cnts}cts.pha'):
            continue
        else:
            source_10cts_pha_name = f'{j}_pc_{cnts}cts.pha'

        # get our nH value    
        nH = runnh(ra, dec)[1]    

        # here is the text we are going to write
        text = ["statistic chi\n",
                f"data 1:1 {source_10cts_pha_name}\n",
#                 f'{outpath}/{name}/spec_files/{j}_pc.arf\n',
#                 f'{outpath}/{name}/spec_files/{j}_bkg.pha\n',
                "ignore bad\n",
                "ignore 1\n",
                'method leven 10 0.01\n',
                'abund angr\n',
                'xsect bcmc\n',
                'cosmo 70 0 0.70\n',
                'systematic 0\n',
                "model  phabs*mekal\n",
                f"\t{nH/1e22}, -1\n",
                f"\t{kT}, -1\n",
                "\t1\n", 
                "\t0.3\n",
                f"\t{z}\n",
                "\t1\n",
                "\t0\n", 
                "fit\n",
                f'cpd {j}_pc_{cnts}cts.ps/cps\n',
                'setplot energy\n',
                'plot ldata\n',
#                 'rescale y 1.0e-4 0.05\n',
#                 f'label file {name}/spec_files/{j}_pc_{cnts}cts.pha',
#                 'plot\n',
#                 'quit\n',
                'cpd none\n',
                "error 1.0 7\n",
                "newpar 1 0\n",
                "fit\n",
                "flux 0.1 2.4\n",
                "flux 0.5 2\n",
                "flux 2 10\n" 
                "exit"
               ]      
        
        #Write it to the script
        with open(f'{outpath}/{name}/spec_files/{j}_pc_{cnts}cts_xspec.in', 'w') as script:
            for line in text:
                script.write(line)

In [11]:
# get file data
data = load_PSZcatalog()
data = data.sort_index(axis=1)

outpath = './data_full'

arr = [{'name':n.replace(' ', '_'), 'outpath':outpath} for n in data['NAME']]
parallel_process(arr, create_bkg_regions, use_kwargs=True, n_jobs=6)
parallel_process(arr, create_src_regions, use_kwargs=True, n_jobs=6)
parallel_process(arr, xselect_pha, use_kwargs=True, n_jobs=6) # 2 hours
parallel_process(arr, mk_arf, use_kwargs=True, n_jobs=6) # 41 minutes
parallel_process(arr, combine_spectra, use_kwargs=True, n_jobs=6)

In [14]:
outpath ='./data_full'
name = 'PSZ2_G075.08+19.83'
# name = 'PSZ2_G139.62+24.18'

In [ ]:
create_bkg_regions(name, outpath)

In [ ]:
create_src_regions(name, outpath)

In [13]:
xselect_pha(name, outpath)

In [16]:
mk_arf(name, outpath)

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


xrtmkarf phafile=./data_full/PSZ2_G075.08+19.83/spec_files/00049935004_1_pc.pha srcx=531.5609397463197 srcy=659.3074947408676 outfile=./data_full/PSZ2_G075.08+19.83/spec_files/00049935004_1_pc.arf psfflag=yes extended=yes expofile=./data_full/PSZ2_G075.08+19.83/reduced/00049935004/sw00049935004xpcw3po_ex.img clobber+
xrtmkarf phafile=./data_full/PSZ2_G075.08+19.83/spec_files/00049935001_1_pc.pha srcx=550.8285707464637 srcy=675.4285690871842 outfile=./data_full/PSZ2_G075.08+19.83/spec_files/00049935001_1_pc.arf psfflag=yes extended=yes expofile=./data_full/PSZ2_G075.08+19.83/reduced/00049935001/sw00049935001xpcw3po_ex.img clobber+
xrtmkarf phafile=./data_full/PSZ2_G075.08+19.83/spec_files/00049935005_1_pc.pha srcx=588.0864048322447 srcy=674.7261696429267 outfile=./data_full/PSZ2_G075.08+19.83/spec_files/00049935005_1_pc.arf psfflag=yes extended=yes expofile=./data_full/PSZ2_G075.08+19.83/reduced/00049935005/sw00049935005xpcw3po_ex.img clobber+
xrtmkarf phafile=./data_full/PSZ2_G075.08+1

In [20]:
combine_spectra(name, outpath)

addascaspec 1.list 1_pc.pha 1_pc.arf 1_bkg.pha
grppha 1_pc.pha 1_pc_20cts.pha comm="bad 1-31 & group min 20 & exit" clob+


In [70]:
mk_xspec_scripts(name, outpath)

In [74]:
# we are gonna need to change directories below
wd = os.getcwd()
os.chdir(f'{outpath}/{name}/spec_files/')

cmd = "xspec - 1_pc_10cts_xspec.in"
args = shlex.split(cmd)
out = ""

with subprocess.Popen(args,
                      stdout=subprocess.PIPE,
                      universal_newlines=True) as proc:

    try:
        proc.wait(timeout=5)
        out = proc.stdout.read()
        #Read the fluxes and norms from the stdout
        photon_flux = {}
        energy_flux = {}

        norms = []

        outlines = out.split('\n')
        for line in outlines:
            if "Model Flux" in line:
                band_low = line.split(" ")[8].split("(")[1]
                band_high = line.split(" ")[10]
                photons = float(line.split(" ")[3])
                band = band_low + "-" + band_high
                photon_flux[band] = photons

                energy = float(line.split(" ")[5].split("(")[1])
                energy_flux[band] = energy
            elif "mekal" in line and "norm" in line:
                #the norm will be two index before the +/- sign

                pm_index = line.split(" ").index("+/-")
                norms.append(line.split(" ")[pm_index - 2])
            else:
                pass

            #Assuming the current script format, this is the one we want
            #This is with PE Absorption
        goodnorm = float(norms[1])

        #Get the index of the line containing the sigmas
        conf_index = outlines.index(" Parameter   Confidence Range (1)")
        sigma_line = outlines[conf_index + 1]
        sigma_low = float(sigma_line.split(" ")[-1].split("(")[1].split(
            ",")[0])
        sigma_high = float(sigma_line.split(" ")[-1].split("(")[1].split(
            ",")[1].split(")")[0])
        sigma_av = (abs(sigma_low) + abs(sigma_high)) / 2

        SN = goodnorm / sigma_av
#         print(out, energy_flux, photon_flux, goodnorm, sigma_av, SN)
    except:
        proc.kill()
        out = proc.stdout.read()
    os.chdir(wd)
    
    xspec_out, en_flux, phot_flux, norm, sigma, SN = out, energy_flux, photon_flux, goodnorm, sigma_av, SN
    
    print("Energy Flux [ergs/cm^2/s]:\n", en_flux, '\n',
      "Photon Flux [photons]:\n", phot_flux, '\n', "norm: ", norm,
      '\n', "st. deviation: ", sigma, '\n', "S/N Ratio: ", SN,
      '\n')

Energy Flux [ergs/cm^2/s]:
 {'0.10000-2.4000': 7.6111e-13, '0.50000-2.0000': 4.6759e-13, '2.0000-10.000': 6.6507e-13} 
 Photon Flux [photons]:
 {'0.10000-2.4000': 0.00087775, '0.50000-2.0000': 0.00029207, '2.0000-10.000': 0.00010802} 
 norm:  0.00116974 
 st. deviation:  0.000113739 
 S/N Ratio:  10.284423109047909 



In [54]:
args

['xspec', '<', './data_full/PSZ2_G075.08+19.83/spec_files/1_pc_10cts_xspec.in']